In [1]:
!rmdir /S /Q lightning_logs

In [2]:
import lightning
import pandas
import torch
import torch.nn
import torchmetrics

In [3]:
lightning.seed_everything(0)

Seed set to 0


0

In [4]:
orig_train_df = pandas.read_csv("train.csv")

In [5]:
train_df = orig_train_df.sample(n=1000, random_state=0)
val_df = orig_train_df.sample(n=500, random_state=0)
test_df = orig_train_df.sample(n=500, random_state=0)

In [6]:
train_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
16275,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19204,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18518,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25780,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16228,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11599,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27526,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28097,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_df_X = train_df.drop(columns=["label"])
train_df_X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
16275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_df_y = train_df["label"]
train_df_y

16275    3
19204    6
18518    9
25780    5
16228    6
        ..
10235    0
11599    5
27526    9
28097    9
41349    2
Name: label, Length: 1000, dtype: int64

In [9]:
val_df_X = val_df.drop(columns=["label"])
val_df_X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
16275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
val_df_y = val_df["label"]
val_df_y

16275    3
19204    6
18518    9
25780    5
16228    6
        ..
40459    9
28155    1
33923    7
32406    5
41155    3
Name: label, Length: 500, dtype: int64

In [11]:
test_df_X = test_df.drop(columns=["label"])
test_df_X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
16275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test_df_y = test_df["label"]
test_df_y

16275    3
19204    6
18518    9
25780    5
16228    6
        ..
40459    9
28155    1
33923    7
32406    5
41155    3
Name: label, Length: 500, dtype: int64

In [13]:
train_tensor_X = torch.tensor(train_df_X.values, dtype=torch.float32)
train_tensor_y = torch.tensor(train_df_y.values, dtype=torch.int64)
train_data = torch.utils.data.TensorDataset(train_tensor_X, train_tensor_y)
train_loader = torch.utils.data.DataLoader(train_data)

In [14]:
val_tensor_X = torch.tensor(val_df_X.values, dtype=torch.float32)
val_tensor_y = torch.tensor(val_df_y.values, dtype=torch.int64)
val_data = torch.utils.data.TensorDataset(val_tensor_X, val_tensor_y)
val_loader = torch.utils.data.DataLoader(val_data)

In [15]:
val_tensor_X.shape

torch.Size([500, 784])

In [16]:
val_tensor_y.shape

torch.Size([500])

In [17]:
test_tensor_X = torch.tensor(test_df_X.values, dtype=torch.float32)
test_tensor_y = torch.tensor(test_df_y.values, dtype=torch.int64)
test_data = torch.utils.data.TensorDataset(test_tensor_X, test_tensor_y)
test_loader = torch.utils.data.DataLoader(test_data)

In [18]:
test_tensor_X.shape

torch.Size([500, 784])

In [19]:
test_tensor_y.shape

torch.Size([500])

In [20]:
class DigitRecognizer(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = torch.nn.Sequential(
            torch.nn.Linear(784, 1024), torch.nn.ReLU(),
            torch.nn.Linear(1024, 10), torch.nn.Softmax(dim=1),
        )
        self._accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self._stack(x)

    def training_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        acc = self._accuracy(y_hat, y)
        return (loss, acc)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

In [21]:
trainer = lightning.Trainer(callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_acc", mode="max")])
dr = DigitRecognizer()
trainer.fit(dr, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | _stack    | Sequential         | 814 K  | train
1 | _accuracy | MulticlassAccuracy | 0      | train
---------------------------------------------------------
814 K     Trainable params
0         Non-trainable params
814 K     Total params
3.256     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

In [22]:
trainer.test(dataloaders=test_loader)

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\hsmtk\repo\fantastic-octo-dollop\digit-recognizer\lightning_logs\version_0\checkpoints\epoch=3-step=4000.ckpt
Loaded model weights from the checkpoint at C:\Users\hsmtk\repo\fantastic-octo-dollop\digit-recognizer\lightning_logs\version_0\checkpoints\epoch=3-step=4000.ckpt
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `n

Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.09600000083446503
        test_loss           2.3651418685913086
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.3651418685913086, 'test_acc': 0.09600000083446503}]

In [23]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 20376), started 0:37:09 ago. (Use '!kill 20376' to kill it.)

In [24]:
problem_df = pandas.read_csv("test.csv")
problem_tensor_X = torch.tensor(problem_df.values, dtype=torch.float32)

In [25]:
with torch.no_grad():
    predicted = dr(problem_tensor_X)

In [26]:
predicted.argmax(axis=1)

tensor([0, 0, 0,  ..., 0, 0, 0])